# Clasificador Naive Bayes de noticias argentinas
El objetivo de este ejercicio es implementar un clasificador de texto utilizando el **clasificador ingenuo de Bayes** sobre el
conjunto de datos *”Noticias Argentinas”* para clasificar cada noticia según su tipo.

Librerías utilizadas:

In [1]:
import pandas as pd
import numpy as np
import nltk
import math

from nltk.corpus import stopwords

## Análisis del dataset

En principio contamos con 164690 tuplas:

In [2]:
df = pd.read_excel("Noticias_argentinas_clean.xlsx")
df

,fecha,titular,fuente,categoria
0,2018-12-13 15:49:06,Se van los Melli,NaN,Noticias destacadas
1,2018-12-26 21:21:41,Cantos racistas en el Calcio,NaN,Noticias destacadas
2,2018-12-26 21:21:41,Cantos racistas en el Calcio,NaN,Noticias destacadas
3,2019-01-13 16:35:30,Los que viajan a Uruguay son...,NaN,Noticias destacadas
4,2019-01-13 16:35:30,Los que viajan a Uruguay son...,NaN,Noticias destacadas
...,...,...,...,...
164685,2018-11-26 11:34:11,River Boca: el Gobierno nacional pide â€œinves...,NaN,NaN
164686,2018-11-26 11:34:11,River Boca: el Gobierno nacional pide â€œinves...,NaN,NaN
164687,2018-11-24 22:25:24,Se postergó San Lorenzo Huracán: el resto de l...,NaN,NaN
164688,2018-11-24 22:25:24,Se postergó San Lorenzo Huracán: el resto de l...,NaN,NaN


In [3]:
df['fuente'].unique().shape

(958,)

Al enumerar las categorías nos encontramos con el valor NaN, indicando que hay tuplas sin especificar su categoría:

In [4]:
df['categoria'].unique()

array(['Noticias destacadas', 'Ciencia y Tecnologia', nan, 'Deportes',
       'Entretenimiento', 'Destacadas', 'Actualidad', 'Crítica'],
      dtype=object)

Al contar los valores por cada una (sin vacíos):

In [5]:
print(df.groupby('categoria')['titular'].count())

categoria
Actualidad                   1
Ciencia y Tecnologia      2966
Crítica                      4
Deportes                  2969
Destacadas                2971
Entretenimiento           2961
Noticias destacadas     133864
Name: titular, dtype: int64


Además, al observar el dataset más arriba, podemos ver que existen tuplas que se encuentran repetidas. Teniendo esto en cuenta, contamos nuevamente las noticias:

In [6]:
print(df.groupby('categoria')['titular'].nunique())

categoria
Actualidad                  1
Ciencia y Tecnologia      710
Crítica                     1
Deportes                 1402
Destacadas               1731
Entretenimiento          1199
Noticias destacadas     39491
Name: titular, dtype: int64


## División del conjunto de textos

Utilizamos el método K-Fold para futura cross-validation:

In [7]:
def k_fold_split(df : pd.DataFrame, k : int) -> tuple[pd.DataFrame, pd.DataFrame]:
    if (k < 2) : raise ValueError("k must be >= 2. The value of k was: {}".format(k))
    df         = df.copy()
    df         = df.sample(frac=1)
    df_size    = df.shape[0]
    fold_times = math.ceil(df_size / k)
    train_df   = pd.DataFrame(columns=df.columns)
    test_df    = pd.DataFrame(columns=df.columns)
    for i in range(fold_times):
        curr_fold = df.iloc[i*k:(i+1)*k]
        train_df  = pd.concat([train_df, curr_fold.iloc[0:k-1]])
        test_df   = pd.concat([test_df,  curr_fold.iloc[k-1].to_frame().T]) \
                    if curr_fold.shape[0] >= k else test_df
    return train_df, test_df

Vamos a crear dos conjuntos, uno de entrenamiento y otro de testeo, por lo que nos queda:

In [8]:
k = 2

In [9]:
test_size = df.shape[0] / k
(test_size * (k-1), test_size)

(82345.0, 82345.0)

In [10]:
train_df, test_df = k_fold_split(df, k)

## ❌ BORRAR SECCIÓN
Achico train_df para que no me explote el notebook

In [11]:
train_df = pd.DataFrame(train_df[:2500])

## Preprocesamiento de los datos

Vamos a expresar los títulos como un array conformado por sus palabras relevantes lematizadas:

In [12]:
nltk.download('stopwords')
stop_words = set(stopwords.words('spanish'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:
import en_core_web_sm
pipeline = en_core_web_sm.load()

In [14]:
def preprocess_text(title_string):
  title = pipeline(title_string.strip('.,+-#:;¿?¡!"\''))
  lemmas = []
  for tok in title:
    word = tok.lemma_.lower()
    if word not in stop_words:
      lemmas.append(word)
  return lemmas

In [15]:
train_df['titular'] = train_df['titular'].apply(preprocess_text)
train_df.head()

,fecha,titular,fuente,categoria
53138,2019-02-24 10:39:00,"[ricado, bochini, explicó, independiente, cayó...",El Intransigente,Noticias destacadas
4863,2018-12-15 08:29:00,"[quinto, sábado, ,, perdió, fuerza, protesta, ...",Ambito.com,Noticias destacadas
101070,2018-11-28 17:00:05,"[habló, amiga, acompañó, pier, fritzsche, luch...",La Voz del Interior,Entretenimiento
60292,2019-01-09 17:22:22,"[fiebre, amarilla, :, ¿, necesaria, dónde, apl...",FILO,Noticias destacadas
152550,2019-08-09 15:09:59,"[curiosidad, mata, gato, ,, indiscretos, cada,...",Tiempo de San Juan,Noticias destacadas


## One-Hot Encoding

Para empezar creamos un set con todas las palabras de nuestro vocabulario, evitando repetidos:

In [16]:
vocab = set()
for title in train_df['titular']:
  vocab.update(title)
vocab = list(vocab)

print(f"Vocab len: {len(vocab)}")

Vocab len: 7021


Por último aplicamos One-Hot Encoding para representar a las palabras del set:

In [17]:
def one_hot_encode(tokens):
    encoding = np.zeros(len(vocab))
    for word in tokens:
        if word in vocab:
            word_index = vocab.index(word)
            encoding[word_index] = 1
    return encoding

In [18]:
train_df['titular_onehot'] = train_df['titular'].apply(one_hot_encode)
train_df.head()

,fecha,titular,fuente,categoria,titular_onehot
53138,2019-02-24 10:39:00,"[ricado, bochini, explicó, independiente, cayó...",El Intransigente,Noticias destacadas,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4863,2018-12-15 08:29:00,"[quinto, sábado, ,, perdió, fuerza, protesta, ...",Ambito.com,Noticias destacadas,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
101070,2018-11-28 17:00:05,"[habló, amiga, acompañó, pier, fritzsche, luch...",La Voz del Interior,Entretenimiento,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
60292,2019-01-09 17:22:22,"[fiebre, amarilla, :, ¿, necesaria, dónde, apl...",FILO,Noticias destacadas,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
152550,2019-08-09 15:09:59,"[curiosidad, mata, gato, ,, indiscretos, cada,...",Tiempo de San Juan,Noticias destacadas,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


Si lo queremos como matriz:

In [19]:
characteristics = train_df['titular_onehot'].array
characteristics_matrix = np.vstack(characteristics.to_numpy())
characteristics_matrix.shape

(2500, 7021)